### Gemini Function Calling

Google's Gemini models support function calling (also known as tool use), which allows the LLM to identify when a function should be called and provide parameters for that function call.

In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [3]:
import google.generativeai as genai
import json
import os

# Configure the Gemini API
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

def chat(query):
    model = genai.GenerativeModel("gemini-2.5-flash-lite")
    response = model.generate_content(query)
    return response.text

query = "How much does pizza salami cost?"
message = chat(query)
message

c:\Users\rishu.kumar\Documents\0011_OFFICE_FOLDER\Udemy Courses\LangChain-Udemy-Course\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\rishu.kumar\AppData\Local\Temp\ipykernel_29504\2135204452.py:1: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


'The cost of pizza salami can vary quite a bit depending on several factors:\n\n**Factors Influencing Pizza Salami Cost:**\n\n* **Type of Salami:**\n    * **Standard Pepperoni:** This is the most common and generally the most affordable.\n    * **Spicy Salami (e.g., Calabrese, Hot Sopressata):** These often have a bit more kick and can be slightly more expensive.\n    * **Gourmet or Artisanal Salami:** Salami made with higher quality ingredients, unique spice blends, or traditional curing methods will be the most expensive. Examples include Finocchiona, Genoa Salami, or specific regional varieties.\n    * **Lower Quality Salami:** Sometimes, very cheap pizzas might use a less flavorful, processed salami.\n\n* **Brand:** Well-known and established brands might command a slightly higher price than generic or store-brand options.\n\n* **Where You Buy It:**\n    * **Supermarket:** You\'ll find a wide range of prices here, from budget-friendly to premium.\n    * **Specialty Deli/Butcher Sho

To make use of Function calling you need:

1. A function
2. A dictionary which describes the function (function declaration)

In [4]:
import json

def get_pizza_info(pizza_name: str):
    """Get name and price of a pizza of the restaurant."""
    pizza_info = {
        "name": pizza_name,
        "price": "10.99",
    }
    return json.dumps(pizza_info)

# Define the function declaration for Gemini
get_pizza_info_declaration = {
    "name": "get_pizza_info",
    "description": "Get name and price of a pizza of the restaurant",
    "parameters": {
        "type": "object",
        "properties": {
            "pizza_name": {
                "type": "string",
                "description": "The name of the pizza, e.g. Salami",
            },
        },
        "required": ["pizza_name"],
    },
}

In [5]:
def chat_with_functions(query):
    model = genai.GenerativeModel(
        "gemini-2.5-flash-lite",
        tools=[{
            "function_declarations": [get_pizza_info_declaration]
        }]
    )
    
    chat = model.start_chat()
    response = chat.send_message(query)
    
    return response, chat

# Test with a non-function query
response, chat = chat_with_functions("What is the capital of france?")
print(response.text)

I can only provide information about pizzas. Would you like to know about any pizza? 



In [6]:
# Test with a function query
query = "How much does pizza salami cost?"
response, chat = chat_with_functions(query)

print("Response:", response)
print("\nFunction call:", response.candidates[0].content.parts[0].function_call)

Response: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "get_pizza_info",
                  "args": {
                    "pizza_name": "Salami"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 67,
        "candidates_token_count": 20,
        "total_token_count": 87
      },
      "model_version": "gemini-2.5-flash-lite"
    }),
)

Function call: name: "get_pizza_info"
args {
  fields {
    key: "pizza_name"
    value {
      string_value: "Salami"
    }
  }
}



In [7]:
# Extract function call and execute
function_call = response.candidates[0].content.parts[0].function_call

if function_call.name == "get_pizza_info":
    pizza_name = function_call.args.get("pizza_name")
    print(f"Pizza name: {pizza_name}")
    
    function_response = get_pizza_info(pizza_name=pizza_name)
    print(f"Function response: {function_response}")

Pizza name: Salami
Function response: {"name": "Salami", "price": "10.99"}


In [9]:
# Send the function result back to the model
response = chat.send_message(
    content={
        "parts": [{
            "function_response": {
                "name": "get_pizza_info",
                "response": json.loads(function_response)
            }
        }]
    }
)

print("Final response:", response.text)

Final response: The Salami pizza costs 10.99.


## Using LangChain with Gemini Function Calling

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, FunctionMessage

# Initialize the LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0)

# Define the tool using the @tool decorator
@tool
def get_pizza_info(pizza_name: str) -> str:
    """Get name and price of a pizza of the restaurant.
    
    Args:
        pizza_name: The name of the pizza, e.g. Salami
    
    Returns:
        A JSON string containing the name and price of the pizza.
    """
    pizza_info = {
        "name": pizza_name,
        "price": "10.99",
    }
    return json.dumps(pizza_info)

tools = [get_pizza_info]

In [11]:
# Bind tools to the LLM
llm_with_tools = llm.bind_tools(tools)

# Create a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can provide information about pizzas."),
    ("human", "{input}")
])

# Create the chain
chain = prompt | llm_with_tools

# Test the chain
result = chain.invoke({"input": "How much does pizza salami cost?"})
print(result)

content='' additional_kwargs={'function_call': {'name': 'get_pizza_info', 'arguments': '{"pizza_name": "Salami"}'}} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c9317-3ad1-7e41-97e8-6db75ee1cd49-0' tool_calls=[{'name': 'get_pizza_info', 'args': {'pizza_name': 'Salami'}, 'id': '7131c55b-97ac-4053-aec4-3f981bd380a7', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 109, 'output_tokens': 20, 'total_tokens': 129, 'input_token_details': {'cache_read': 0}}


In [12]:
# Check if the model wants to call a function
if result.tool_calls:
    print("Tool calls:", result.tool_calls)
    
    # Execute the function
    for tool_call in result.tool_calls:
        if tool_call["name"] == "get_pizza_info":
            function_response = get_pizza_info.invoke(tool_call["args"])
            print(f"Function response: {function_response}")

Tool calls: [{'name': 'get_pizza_info', 'args': {'pizza_name': 'Salami'}, 'id': '7131c55b-97ac-4053-aec4-3f981bd380a7', 'type': 'tool_call'}]
Function response: {"name": "Salami", "price": "10.99"}


In [13]:
# Create a more complete chain with function execution
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import ToolMessage

def execute_tools(msg):
    """Execute tool calls from the model's response."""
    tool_results = []
    for tool_call in msg.tool_calls:
        if tool_call["name"] == "get_pizza_info":
            result = get_pizza_info.invoke(tool_call["args"])
            tool_results.append(
                ToolMessage(
                    content=result,
                    tool_call_id=tool_call["id"]
                )
            )
    return tool_results

# Chain that handles function calling
def chat_with_tools(query):
    # First call
    messages = [
        HumanMessage(content=query)
    ]
    
    response = llm_with_tools.invoke(messages)
    
    # Check if function calling is needed
    if response.tool_calls:
        # Execute functions
        tool_results = execute_tools(response)
        
        # Add function call and results to messages
        messages.append(response)
        messages.extend(tool_results)
        
        # Get final response
        final_response = llm_with_tools.invoke(messages)
        return final_response
    
    return response

# Test
result = chat_with_tools("How much does pizza salami cost?")
print("Final result:", result.content)

Final result: The Salami pizza costs 10.99.


## Using Pydantic Classes for Tool Definitions

In [14]:
from pydantic import BaseModel, Field
from typing import Optional
import json

# Define a separate function for use with StructuredTool.from_function()
# Note: We cannot use get_pizza_info here because it's already decorated with @tool
def get_pizza_info_basic(pizza_name: str) -> str:
    """Get name and price of a pizza of the restaurant."""
    pizza_info = {
        "name": pizza_name,
        "price": "10.99",
    }
    return json.dumps(pizza_info)


class GetPizzaInfo(BaseModel):
    """Get name and price of a pizza of the restaurant."""
    
    pizza_name: str = Field(..., description="The name of the pizza, e.g. Salami")


# Convert Pydantic class to tool
from langchain_core.tools import StructuredTool

get_pizza_info_tool = StructuredTool.from_function(
    func=get_pizza_info_basic,
    name="get_pizza_info",
    description="Get name and price of a pizza of the restaurant",
    args_schema=GetPizzaInfo
)

tools_from_pydantic = [get_pizza_info_tool]

# Bind to LLM
llm_with_pydantic_tools = llm.bind_tools(tools_from_pydantic)

# Test
result = llm_with_pydantic_tools.invoke("How much does pizza salami cost?")
print(result)

content='' additional_kwargs={'function_call': {'name': 'get_pizza_info', 'arguments': '{"pizza_name": "Salami"}'}} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c9317-6c6a-7f70-9d6b-00128c576347-0' tool_calls=[{'name': 'get_pizza_info', 'args': {'pizza_name': 'Salami'}, 'id': 'bae0f4a8-d4d6-4330-8018-c75226daa9f9', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 67, 'output_tokens': 20, 'total_tokens': 87, 'input_token_details': {'cache_read': 0}}


In [15]:
# Execute if tool calls are present
if result.tool_calls:
    for tool_call in result.tool_calls:
        if tool_call["name"] == "get_pizza_info":
            output = get_pizza_info_tool.invoke(tool_call["args"])
            print(f"Tool output: {output}")

Tool output: {"name": "Salami", "price": "10.99"}
